In [1]:
!pip install transformers
!pip install tokenizers
!pip install nltk
!pip install pandas
!pip install scipy
!pip install scikit-learn

In [86]:
!pip install ekphrasis

In [87]:
import pandas as pd
df = pd.read_csv('/content/emoticon13.csv')

df.head()

total_emojis =[]

emoji_dictionary = pd.DataFrame(columns=['emoji', 'meaning', 'sentiment'])

for i in range(len(df['icons'])):
  str_list_of_emojis = df['icons'][i]
  str_list_of_emojis = str_list_of_emojis[1:-1]
  #print(str_list_of_emojis)
  list_of_emojis = str_list_of_emojis.split(",")
  #print(list_of_emojis)
  list_of_emojis = [emoji.strip() for emoji in list_of_emojis]
  #print(list_of_emojis)
  list_of_emojis = [emoji[1:-1] for emoji in list_of_emojis]
  #print(list_of_emojis)
  for emoji in list_of_emojis:
    emoji_dictionary.loc[len(emoji_dictionary)] = [emoji, " "+df['meaning'][i].lower()+" ", df['Sentiment'][i]]

  total_emojis.extend(list_of_emojis)
print(total_emojis)

[':-]', '8-)', '8-)', '=]', '^^', '^ . ^', ':-)', '><', '>_<', ':)', ':-D', ':D', '=D', ":')", 'xd', 'x-D', 'xD', ':-))', ':))', ':-((', ':((', ':{', ':@', ':(', ":'(", ":'-D", ":'D", ':<', ':O', 'x3', ':3', ';]', ';D', 'xp', ':-p', ':p', 'd:', '=p', ':-/', ':/', '=/', ':-|', ':|', '8-)', '8)', '</3', '<3', 'o7', '-.-', 'qq', 'x_x', 'o-o', 'o_o', ' - _ - )', ' - __ - )', ' . _ . )']


In [88]:
emoji_dictionary

,emoji,meaning,sentiment
0,:-],happy,1
1,8-),happy,1
2,8-),happy,1
3,=],happy,1
4,^^,happy,1
5,^ . ^,happy,1
6,:-),happy,1
7,><,happy,1
8,>_<,happy,1
9,:),happy,1


In [89]:

emojis_with_parentheses = []
for emoji in total_emojis:
  if '(' in emoji or ')' in emoji:
    if emoji != ":)" and emoji != ":(" and emoji != ":((" and emoji != ":))":
      emojis_with_parentheses.append(emoji)
print(emojis_with_parentheses)

['8-)', '8-)', ':-)', ":')", ':-))', ':-((', ":'(", '8-)', '8)', ' - _ - )', ' - __ - )', ' . _ . )']


In [136]:
import numpy as np
import pandas as pd

with open("train_neg.txt") as f:
    neg_txt = [line for line in f]
    neg_df = pd.DataFrame(neg_txt, columns=['line']).drop_duplicates(keep='first')
    print(neg_df.head())
f.close()

with open("train_pos.txt") as f:
    pos_txt = [line for line in f]
    pos_df = pd.DataFrame(pos_txt, columns=['line']).drop_duplicates(keep='first')
    print(pos_df.head())
f.close

pos_df['label'] = 1
neg_df['label'] = -1

full_df_orig= pd.concat([pos_df,neg_df],ignore_index=True)
full_df_orig.tail()

                                                line
0  vinco tresorpack 6 ( difficulty 10 of 10 objec...
1  glad i dot have taks tomorrow ! ! #thankful #s...
2  1-3 vs celtics in the regular season = were fu...
3  <user> i could actually kill that girl i'm so ...
4  <user> <user> <user> i find that very hard to ...
                                                line
0  <user> i dunno justin read my mention or not ....
1  because your logic is so dumb , i won't even c...
2  " <user> just put casper in a box ! " looved t...
3  <user> <user> thanks sir > > don't trip lil ma...
4  visiting my brother tmr is the bestest birthda...


,line,label
181316,can't wait to fake tan tonight ! hate being pa...,-1
181317,<user> darling i lost my internet connection ....,-1
181318,kanguru defender basic 4 gb usb 2.0 flash driv...,-1
181319,rizan is sad now\n,-1
181320,"no text back ? yea , he mad\n",-1


In [137]:
import re
import nltk
from tqdm import tqdm
corpus_orig = []
for i in tqdm(range(len(full_df_orig))):
  #review = re.sub('[^a-zA-Z]', ' ', full_df['line'][i])
  #review = review.lower()
  #review = review.split()
  #review = ' '.join(review)
  review = full_df_orig['line'][i]
  review = review.lower()
  corpus_orig.append(review)

100%|██████████| 181321/181321 [00:01<00:00, 91808.62it/s]


## Remove twitter specific tags with parenthesis that doesn't help with recovering the labels

In [138]:

def twitter_specific_analysis(line,num_replaced_lines):

  replaced = False
  if "( <user> live on <url>" in line:
    line = line.replace("( <user> live on <url>", "")
    replaced = True

  if "( <user> <url>" in line:
    line = line.replace("( <user> <url>", "")
    replaced = True

  if "( <url>" in line:
    line = line.replace("( <url>","")
    replaced = True
  if "( live at <url>" in line:
    line = line.replace("( live at <url>","")
    replaced = True

  if "( rt <user>" in line:
    line = line.replace("( rt <user>","")
    replaced = True

  if "<url> via <user>" in line:
    line = line.replace("<url> via <user>","")
    replaced = True
  if "( via <user> <url>" in line:
    line = line.replace("( via <user> <url>","")
    replaced = True

  if "( via <user>" in line:
    line = line.replace("( via <user>","")
    replaced = True

  if ") ) )" in line:
    line = line.replace(") ) )","excited")
    replaced = True

  if "< ^ , ^ )" in line:
    line = line.replace("< ^ , ^ )","happy")
    replaced = True

  # Only replace emojis with parenthesis that can mess with the prediction
  for emoji in emojis_with_parentheses:
    if emoji in line:
      row = emoji_dictionary.loc[emoji_dictionary['emoji'] == emoji]
      replacement = row['meaning'].values[0]
      line = line.replace(emoji, replacement)
      replaced = True
  if replaced ==True:
    num_replaced_lines+=1
  return line,num_replaced_lines


## Sometimes users open a parenthesis and doesn't close this - also exclude those cases - More cases of this in negative tweets though

In [139]:
def detect_single_parenthesis(s):
  if s.count('(') == 1 and s.count(')')==0:
    return True

## Spam Detection

In [166]:
def spam_detection(line):
  counts = line.count("<number>")
  if "paperback" in line and "<url>" in line :
      return True
  elif "hardcover" in line and "<url>" in line :
      return True
  elif "academic edition" in line and "<url>" in line :
      return True
  elif "<url>" in line and counts>1:
      return True
  else:
    return False

## Detect and Remove parenthesis

In [141]:
def detect_dyck_language(s):
    # Check if the string contains only a single opening parenthesis "("
    if s.count('(') == 1 and s.count(')')==0:
        s = s.replace("(","")
        return s, None

    stack = []
    for index, char in enumerate(s):
        if char == '(':
            stack.append((char, index))
        elif char == ')':
            if not stack:
                return s,(index, ')')
            stack.pop()

    if stack:
        _, index = stack.pop()
        return s,(index, '(')
    else:
        return s,None

## Step1: Calculate emoji scores

In [142]:

labels = full_df_orig['label']
emoji_scores = []
corpus_after_emoji_replacement = []

for index, line in enumerate(corpus_orig):
    words = set(line.replace("\n","").split(" "))

    emoji_score =0
    present_emoji = words & set(total_emojis)
    if len(present_emoji) > 0:
        for emoji in present_emoji:

            row = emoji_dictionary.loc[emoji_dictionary['emoji'] == emoji]
            replacement = row['meaning'].values[0]

            line_after_emoji_replacement = line.replace(emoji, replacement)

            if emoji_dictionary[emoji_dictionary['emoji'] == emoji]['sentiment'].values[0] == 1:
              emoji_score += 1

            elif emoji_dictionary[emoji_dictionary['emoji'] == emoji]['sentiment'].values[0] == 0:
              emoji_score -= 1
    else:
      line_after_emoji_replacement = line

    corpus_after_emoji_replacement.append(line_after_emoji_replacement)

    if emoji_score > 0:
      emoji_score = 1
    elif emoji_score < 0:
      emoji_score = -1
    else:
      emoji_score = 0
    emoji_scores.append(emoji_score)

print(len(emoji_scores))

181321


In [143]:

print(len(corpus_after_emoji_replacement))

181321


In [144]:
df_with_emoji_scores = full_df_orig.copy()
df_with_emoji_scores['emoji_scores'] = emoji_scores
df_with_emoji_scores['line'] = corpus_after_emoji_replacement

df_with_emoji_scores[df_with_emoji_scores['emoji_scores']!=0].head(10)

,line,label,emoji_scores
6,<user> #1dnextalbumtitle : feel for you / roll...,1,1
21,i have the worlds best dad . love \n,1,1
30,<user> <user> lol chloe laughing ill teach y...,1,1
36,"#ff to my good , hilarious , sweet and kind fr...",1,1
52,<user> i haven't seen myself though and they d...,1,1
72,<user> hehe love well hello there\n,1,1
78,"<user> aww , you are a very sweet kind . smurf...",1,1
140,<user> are you getting that tattoo ? ? love \n,1,1
160,<user> could you please follow me ? ? if you d...,1,1
169,"when a girl can take jokes , make you laugh , ...",1,1


In [145]:
len(df_with_emoji_scores[df_with_emoji_scores['emoji_scores']!=0])

6545

## Evaluation

In [146]:
eval_emoji_score = df_with_emoji_scores[df_with_emoji_scores['emoji_scores']!=0]
wrongs = df_with_emoji_scores[(df_with_emoji_scores['emoji_scores']!=0) & (df_with_emoji_scores['emoji_scores']!=df_with_emoji_scores['label'])]
len(wrongs)

2002

In [147]:
len( df_with_emoji_scores[(df_with_emoji_scores['emoji_scores']!=0)])

6545

In [148]:
wrongs.head(10)

,line,label,emoji_scores
890,<user> i know ! unhappy hey did you know tha...,1,-1
1597,<user> okey thank you so much ! i'm going to w...,1,-1
1797,<user> unhappy and yeah i tell her as much a...,1,-1
1933,im not up for this unhappy soz unhappy \n,1,-1
2491,<user> thanks i'm feeling a little better not ...,1,-1
3425,going to bring this little fur ball back to it...,1,-1
3821,allison & rob get along too fuckin good i dont...,1,-1
3897,"<user> omg , so attractive image in your head ...",1,-1
3953,anyone want to take me with to tomorrowland ? ...,1,-1
4020,"seriously , have to start studying for test al...",1,-1


## Sometimes emojis translations are too "radical"

## Replace numbers for spam detection

In [167]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import re

def process_string(input_string:str):
    # Find all hashtags
    hashtags = re.findall(r'<hashtag>(.*?)</hashtag>', input_string)

    # Split the string by hashtags
    parts = re.split(r'<hashtag>.*?</hashtag>', input_string)

    # Remove empty strings from parts
    parts = [part.strip() for part in parts if part.strip()]

    # Format the output list
    hashtags_written_out = input_string.replace('<hashtag> ', '').replace('</hashtag> ', '')
    no_hashtags = ''.join(parts) if '<hashtag>' in input_string else ''
    output = [hashtags_written_out] + [no_hashtags] + [hashtags]

    return output

def preprocess(tweets:list):

    text_processor = TextPreProcessor(
        fix_html=True,
        segmenter="twitter",
        corrector="twitter",
        tokenizer=SocialTokenizer(lowercase=True).tokenize,
        #dicts=[emoticons],
        unpack_hashtags=True,
        unpack_contractions=True,
        spell_correct_elong=True,
        normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
        annotate={'hashtag','number'}
    )

    pre_process_1 = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in tweets]
    pre_process_2 = [process_string(tweet) for tweet in pre_process_1]

    return pre_process_2

In [150]:
res = preprocess(corpus_after_emoji_replacement)
corpus_with_no_num = []
for triple in res:
  corpus_with_no_num.append(triple[0])
count =0
count1 = 0
#CHECK IF IT DID THE JOB
for i in range(len(corpus_with_no_num)):
    line = corpus_with_no_num[i]
    s,first = detect_dyck_language(line)

    if first is not None:
      count+=1

print(f"number of emojis that will be put back {count}")


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...
number of emojis that will be put back 10260


In [151]:
len(corpus_with_no_num) == len(corpus_after_emoji_replacement) ==181321

True

In [152]:

for i in range(100):
  if corpus_with_no_num[i] != corpus_after_emoji_replacement[i]:
    print(corpus_after_emoji_replacement[i])
    print(corpus_with_no_num[i])




<user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me #believe 15

<user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me believe <number>
because your logic is so dumb , i won't even crop out your name or your photo . tsk . <url>

because your logic is so dumb , i will not even crop out your name or your photo . tsk . <url>
" <user> just put casper in a box ! " looved the battle ! #crakkbitch

" <user> just put casper in a box ! " looved the battle ! crakk bitch </hashtag>
<user> <user> thanks sir > > don't trip lil mama ... just keep doin ya thang !

<user> <user> thanks sir > > do not trip lil mama . . . just keep doin ya thang !
visiting my brother tmr is the bestest birthday gift eveerrr ! ! !

visiting my brother tmr is the bestest birthday gift eveerrr ! ! !
<user> yay ! ! #lifecompleted . tweet / facebook me to let me know please

<user> yay ! ! life comple

In [168]:
spam_score = []
new_corpus_with_smileys = []

#STATS
num_replaced_lines=0
number_of_dyck = 0
pos_emoji_added = 0
neg_emoji_added = 0

for i, line in enumerate(corpus_with_no_num):

  line1,num_replaced_lines = twitter_specific_analysis(line,num_replaced_lines)

  # do spam detection
  score = spam_detection(line1)

  if score:
    spam_pred = 1
  else:
    spam_pred = 0
  spam_score.append(spam_pred)


  # Delete first parenthesis and Insert fake label
  line2,res = detect_dyck_language(line1)

  if res is not None:
    number_of_dyck+=1
    index,char = res

    if char == '(':
      substring = ':('
      newline = line2[:index] + substring + line2[index:]
      new_corpus_with_smileys.append(newline)
      neg_emoji_added+=1
      #print(newline)
    elif char ==')':
      substring = ':)'
      newline = line2[:index] + substring + line2[index:]
      new_corpus_with_smileys.append(newline)
      pos_emoji_added+=1
      #print(newline)
  else:
    new_corpus_with_smileys.append(line2)

print(f"len(spam_score): {len(spam_score)}")
print(f"len(new_corpus_with_smileys): {len(new_corpus_with_smileys)}")
print(f"new_corpus_with_smileys[:5]: {new_corpus_with_smileys[:5]}")
print(f"pos_emoji_added: {pos_emoji_added}")
print(f"neg_emoji_added: {neg_emoji_added}")
print(f"number_of_total_fake_lables: {number_of_dyck}")
print(f"num_replaced_lines: {num_replaced_lines}")


len(spam_score): 181321
len(new_corpus_with_smileys): 181321
new_corpus_with_smileys[:5]: ['<user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me believe <number>', 'because your logic is so dumb , i will not even crop out your name or your photo . tsk . <url>', '" <user> just put casper in a box ! " looved the battle ! crakk bitch </hashtag>', '<user> <user> thanks sir > > do not trip lil mama . . . just keep doin ya thang !', 'visiting my brother tmr is the bestest birthday gift eveerrr ! ! !']
pos_emoji_added: 4272
neg_emoji_added: 5348
number_of_total_fake_lables: 9620
num_replaced_lines: 2210


In [169]:
full_df_mid = df_with_emoji_scores.copy()
full_df_mid['line'] = new_corpus_with_smileys
full_df_mid['spam_score'] = spam_score
full_df_mid[full_df_mid['spam_score']>0].head()

,line,label,emoji_scores,spam_score
1972,<user> rt pls sell shini star lipbalm idr . <n...,1,0,1
3871,still only <number> away from <number> ! ! sig...,1,0,1
4133,check this out : battery - biz - generic <numb...,1,0,1
4752,casting call for a free shoot for an up and co...,1,0,1
5056,"after <number> years , <number> fans , we fina...",1,0,1


##Evaluation

In [170]:
right = len(full_df_mid[(full_df_mid['spam_score']==1) & (full_df_mid['label']==-1)])
wrong = len(full_df_mid[(full_df_mid['spam_score']==1) & (full_df_mid['label']==1)])
print(f'right samples{right}')
print(f'accuracy {right/(right+wrong)}')

right samples11690
accuracy 0.9883327696990193


In [171]:
full_df_mid[(full_df_mid['spam_score']==1) & (full_df_mid['label']==1)].head()

,line,label,emoji_scores,spam_score
1972,<user> rt pls sell shini star lipbalm idr . <n...,1,0,1
3871,still only <number> away from <number> ! ! sig...,1,0,1
4133,check this out : battery - biz - generic <numb...,1,0,1
4752,casting call for a free shoot for an up and co...,1,0,1
5056,"after <number> years , <number> fans , we fina...",1,0,1


## If i only run the spam test on things with unclosed parenthesis - i only get 5822 samples, although accuracy 1.0

## If without the parenthesis: & count >2
 "right samples7464
accuracy 0.9948020791683326"
## without parenthesis conditon, count>1
right samples11690
accuracy 0.9883327696990193

## Add fake_lables score

In [179]:
corpus_for_fake_lables = full_df_mid['line']
emoji_scores = [score for score in full_df_mid['emoji_scores']]
fake_labels = []
final_corpus_replaced_fake_label = []
new_emoji_scores =[]

#Stats
num_new_labels = 0
num_replaced_lables =0
for index, line in enumerate(corpus_for_fake_lables):
    pred_fake_lables = 0
    if ':)' in line:
        pred_fake_lables = 1
        num_new_labels+=1
    elif ':(' in line:
        pred_fake_lables = -1
        num_new_labels+=1
    else:
        pred_fake_lables = 0

    fake_labels.append(pred_fake_lables)
    #Replace all the added lables
    words = set(line.replace("\n","").split(" "))
    emoji_score = emoji_scores[index]
    present_emoji = words & set([":))",":(("])
    if len(present_emoji) > 0:
        for emoji in present_emoji:
            row = emoji_dictionary.loc[emoji_dictionary['emoji'] == emoji]
            replacement = row['meaning'].values[0]
            line = line.replace("\n","").replace(emoji, replacement)
            if emoji_dictionary[emoji_dictionary['emoji'] == emoji]['sentiment'].values[0] == 1:
              emoji_score += 1

            elif emoji_dictionary[emoji_dictionary['emoji'] == emoji]['sentiment'].values[0] == 0:
              emoji_score -= 1
        num_replaced_lables +=1
    final_corpus_replaced_fake_label.append(line)

    if emoji_score > 0:
      emoji_score = 1
    elif emoji_score < 0:
      emoji_score = -1
    else:
      emoji_score = 0
    new_emoji_scores.append(emoji_score)

print(f'number of fake labels added {num_new_labels}')
print(f'number of replaced lables{num_replaced_lables}')


number of fake labels added 9620
number of replaced lables9479


In [180]:
full_df = full_df_mid.copy()
full_df['fake_lables'] = fake_labels
new_corpus_lower = [line.lower() for line in final_corpus_replaced_fake_label]

full_df['line'] = new_corpus_lower
full_df['emoji_scores'] = new_emoji_scores
full_df.tail(10)

,line,label,emoji_scores,spam_score,fake_lables
181311,"rip meridith , we will miss u . dr . meridith ...",-1,0,0,0
181312,<user> ofcos honey ! but i am not meeting you ...,-1,0,0,0
181313,bedtime have fun at the game tomorrow all you ...,-1,0,0,0
181314,went through my pictures in my phone - & found...,-1,0,0,0
181315,everyday negotiation : navigating the hidden a...,-1,0,1,0
181316,can not wait to fake tan tonight ! hate being ...,-1,0,0,0
181317,<user> darling i lost my internet connection ....,-1,0,0,0
181318,kanguru defender basic <number> gb usb <number...,-1,0,1,0
181319,rizan is sad now,-1,0,0,0
181320,"no text back ? yea , he mad",-1,0,0,0


In [181]:
filtered_df = full_df[full_df['emoji_scores'] > 0]

filtered_df.head(10)

,line,label,emoji_scores,spam_score,fake_lables
6,<user> 1 d next album title : feel for you / r...,1,1,0,0
21,i have the worlds best dad . love,1,1,0,0
26,my summer days : <number> . very happy work ...,1,1,0,1
30,<user> <user> lol chloe laughing ill teach you...,1,1,0,0
34,<user> thank you again love & & i def will get...,1,1,0,1
36,"ff to my good , hilarious , sweet and kind fri...",1,1,0,0
37,<user> apparently not very happy,1,1,0,1
52,<user> i have not seen myself though and they ...,1,1,0,0
57,<user> watching now hehe very happy ),1,1,0,1
62,"<user> aahh , sorry love . . . but i know you ...",1,1,0,1


In [182]:
filtered_df.tail(10)

,line,label,emoji_scores,spam_score,fake_lables
180810,"( ) rt <user> yessir < - , - very happy rt <...",-1,1,0,1
180830,<user> bet your mom was freaking out ! ! ! lol...,-1,1,0,0
180958,"aaawwh how adorable laughing rt "" <user> i mis...",-1,1,0,0
180960,<user> aww hope you do get them i would cry fo...,-1,1,0,0
180985,<user> hello bffl love and idk i went on after...,-1,1,0,0
180991,"everyones crying in class today , we all miss ...",-1,1,0,0
181076,rt <user> no thank you ! ! rt <user> everyone ...,-1,1,0,0
181152,<user> wanted to see the full video . . you li...,-1,1,0,0
181248,"<user> if they will do it , say to bella ! lau...",-1,1,0,0
181309,<user> its not problem follow <user> . . its p...,-1,1,0,0


## Evaluation

In [192]:
filtered_df_final = full_df
# Count based on the provided conditions
count_label_1_and_both_1 = len(filtered_df_final[(filtered_df_final['label'] == 1) & (filtered_df_final['emoji_scores'] == 1) & (filtered_df_final['fake_lables'] == 1)])
count_label_1_and_either_1 = len(filtered_df_final[(filtered_df_final['label'] == 1) & ((filtered_df_final['emoji_scores'] == 1) | (filtered_df_final['fake_lables'] == 1))])
count_label_1_and_neither_1 = len(filtered_df_final[(filtered_df_final['label'] == 1) & (filtered_df_final['emoji_scores'] != 1) & (filtered_df_final['fake_lables'] != 1)])

count_label_0_and_both_minus_1 = len(filtered_df_final[(filtered_df_final['label'] == -1) & (filtered_df_final['emoji_scores'] == -1) & (filtered_df_final['fake_lables'] == -1)])
count_label_0_and_either_minus_1 = len(filtered_df_final[(filtered_df_final['label'] == -1) & ((filtered_df_final['emoji_scores'] == -1) | (filtered_df_final['fake_lables'] == -1))])
count_label_0_and_neither_minus_1 = len(filtered_df_final[(filtered_df_final['label'] == -1) & (filtered_df_final['emoji_scores'] != -1) & (filtered_df_final['fake_lables'] != -1)])

count_no_signal = len(filtered_df_final[(filtered_df_final['emoji_scores'] == 0) & (filtered_df_final['fake_lables'] == 0)& (filtered_df_final['spam_score'] == 0)])
print("Count of rows with label 1 and both emoji_score and fake_labels are 1:", count_label_1_and_both_1)
print("Count of rows with label -1 and both emoji_score and fake_labels are -1:", count_label_0_and_both_minus_1)
print()

print("Count of rows with label 1 and either emoji_score or fake_labels is 1:", count_label_1_and_either_1)
print("Count of rows with label 1 and neither emoji_score nor fake_labels is 1:", count_label_1_and_neither_1)
print("Count of rows with label -1 and either emoji_score or fake_labels is -1:", count_label_0_and_either_minus_1)
print("Count of rows with label -1 and neither emoji_score nor fake_labels is -1:", count_label_0_and_neither_minus_1)

print()


Count of rows with label 1 and both emoji_score and fake_labels are 1: 3791
Count of rows with label -1 and both emoji_score and fake_labels are -1: 5212

Count of rows with label 1 and either emoji_score or fake_labels is 1: 7552
Count of rows with label 1 and neither emoji_score nor fake_labels is 1: 82681
Count of rows with label -1 and either emoji_score or fake_labels is -1: 5878
Count of rows with label -1 and neither emoji_score nor fake_labels is -1: 85210



In [193]:
all = count_label_1_and_either_1+count_label_1_and_neither_1+count_label_0_and_either_minus_1+count_label_0_and_neither_minus_1
print(all)

181321


In [194]:
print(f"percentage of data that doesn't have any fake labels or emojis {(count_label_1_and_neither_1+count_label_0_and_neither_minus_1)/all} ")

percentage of data that doesn't have any fake labels or emojis 0.9259324623182091 


In [196]:
print("data with no signal , no spam, no emoji, no label recovery ",count_no_signal/all)

data with no signal , no spam, no emoji, no label recovery  0.8593213141334981


## Accuracy of fake labels

In [184]:
count_fake_label_right= len(filtered_df_final[(filtered_df_final['fake_lables'] == 1)& (filtered_df_final['label'] == 1)])
count_fake_label_all= len(filtered_df_final[(filtered_df_final['fake_lables'] == 1)])
print(count_fake_label_right)
print(count_fake_label_all)
print(count_fake_label_right/count_fake_label_all)

count_fake_label_right_neg= len(filtered_df_final[(filtered_df_final['fake_lables'] == -1)& (filtered_df_final['label'] == -1)])
count_fake_label_all_neg= len(filtered_df_final[(filtered_df_final['fake_lables'] == -1)])
print(count_fake_label_right_neg)
print(count_fake_label_all_neg)
print(count_fake_label_right_neg/count_fake_label_all_neg)


3874
4272
0.9068352059925093
5237
5348
0.9792445774121167


## Accuracy of spam scores

In [190]:
count_fake_label_right= len(filtered_df_final[(filtered_df_final['spam_score'] == 1)& (filtered_df_final['label'] == -1)])
count_fake_label_all= len(filtered_df_final[(filtered_df_final['spam_score'] == 1)])
print(count_fake_label_right)
print(count_fake_label_all)
print(count_fake_label_right/count_fake_label_all)


11690
11828
0.9883327696990193


## Accuracy of emoji

In [191]:
count_fake_label_right= len(filtered_df_final[(filtered_df_final['emoji_scores'] == 1)& (filtered_df_final['label'] == 1)])
count_fake_label_all= len(filtered_df_final[(filtered_df_final['emoji_scores'] == 1)])
print(count_fake_label_right)
print(count_fake_label_all)
print(count_fake_label_right/count_fake_label_all)

count_fake_label_right_neg= len(filtered_df_final[(filtered_df_final['emoji_scores'] == -1)& (filtered_df_final['label'] == -1)])
count_fake_label_all_neg= len(filtered_df_final[(filtered_df_final['emoji_scores'] == -1)])
print(count_fake_label_right_neg)
print(count_fake_label_all_neg)
print(count_fake_label_right_neg/count_fake_label_all_neg)

7469
9459
0.7896183528914261
5853
6241
0.9378304758852748


## Further removing of parenthesis or other things

In [202]:
final_corpus = []
corpus_before = [line for line in full_df['line']]
for i in range(len(corpus_before)):
  preprocessed_line = corpus_before[i]
  pattern = r'[^a-zA-Z<>.,!? ]'
  # Use re.sub() to replace the matched characters with an empty string
  preprocessed_line = re.sub(pattern, '', preprocessed_line)
  preprocessed_line = preprocessed_line.split()
  preprocessed_line = ' '.join(preprocessed_line)
  final_corpus.append(preprocessed_line)

In [203]:


for i in range(100):
  a = corpus_before[i]
  b = final_corpus[i]
  if a != b:
    print(a)
    print(b)


" <user> just put casper in a box ! " looved the battle ! crakk bitch </hashtag>
<user> just put casper in a box ! looved the battle ! crakk bitch <hashtag>
<user> yay ! ! life completed . tweet / facebook me to let me know please
<user> yay ! ! life completed . tweet facebook me to let me know please
<user> 1 d next album title : feel for you / rollercoaster of life . song cocept : life , yolo , becoming famous ? love <number> follow me plz ! love x15
<user> d next album title feel for you rollercoaster of life . song cocept life , yolo , becoming famous ? love <number> follow me plz ! love x
workin hard or hardly workin rt <user> at hardee ' s with my future coworker <user>
workin hard or hardly workin rt <user> at hardee s with my future coworker <user>
<user> anddd to cheer national s2013 ?
<user> anddd to cheer national s ?
we send an invitation to shop on - line ! here you will find everything you need - without leaving home . . . <url>
we send an invitation to shop on line ! her

In [204]:
full_df['line'] = final_corpus

In [205]:
full_df.to_csv("data_frame_with_semilables_more_preproc6.csv", index=False)